<a href="https://colab.research.google.com/github/yashna02/MRIO-SDA-using-PyMRIO/blob/main/SDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MRIO-SDA analysis on India's CO2 emissions using Python package PyMRIO**

Refer: [PyMRIO Guide](https://pymrio.readthedocs.io/en/latest/index.html)


# **1. Setting Up & Initializing**




In [ ]:
pip install git+https://github.com/IndEcol/pymrio@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/IndEcol/pymrio (to revision master) to /tmp/pip-req-build-8fshnoaz
  Running command git clone --filter=blob:none --quiet https://github.com/IndEcol/pymrio /tmp/pip-req-build-8fshnoaz
  Resolved https://github.com/IndEcol/pymrio to commit a409ecdc5057560c1bf1e0bb1d908537c07f9938
  Preparing metadata (setup.py) ... done
  Created wheel for pymrio: filename=pymrio-0.5.0.dev0-py3-none-any.whl size=133369 sha256=871403cf4b23fca9f30cc867ae0a91653689d19be183f3a573f08f711d2f65ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-lodx0k7s/wheels/0b/7b/34/827ad8ecf1b3e5bfc226dfeecd47b9e1dd6c85eb1880669ea8
Successfully built pymrio


In [ ]:
import pymrio
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# **2. First, consider a single year 2000**

* For better understanding of how PyMRIO works, let us first look at only a single year, 2000.

## **2.1. Importing Required Data (Z, Y, V, F)**

* Source: 
  1. World Input Output Table for 2000: [WIOD 2016 Release](https://www.rug.nl/ggdc/valuechain/wiod/wiod-2016-release)
  2. CO2 Emissions from Environmental Accounts WIOD 2016: [E-WIOD 2016](https://joint-research-centre.ec.europa.eu/document/download/b572c87b-a2fb-4ab6-af38-ff0451273e9e_en?filename=co2em56.zip)

* For full mathematical background: [Math Background](https://pymrio.readthedocs.io/en/latest/math.html)

* m = no. of countries in the dataset = 43 countries in our case (excluding 'NLD' because its emissions data is NA in E-WIOD)

* n = no. of sectors in the dataset = 56 sectors in our case



### **2.1.1. Transaction Matrix (Z)**

* Global inter-industries flows within and across countries.
* Z(i,j) describes the trade from region i to region j for each industry.
* Unit: Millions of USD
* Dimension: (mn x mn) : (2408 x 2408) in our case

* Preceeding Steps:
  * Data was first deflated and converted into constant prices with 2010 as the base year. Refer: [Code](https://github.com/GICN/Deflating_WIOD_Tables/blob/master/code/Deflate_tables.R)
  * The long table so obtained was converted back into a matrix form.  Refer: [Code](https://github.com/yashna02/Extracting_Matrices_from_WIOD_for_MRIO)

In [ ]:
# Load the data
Z_00 = pd.read_excel("/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/Z_2000.xlsx", header=0, index_col=0)

In [ ]:
# Ensure that the dimension is (mn x mn) 
Z_00.shape

(2408, 2408)

In [ ]:
# Make ordered lists of all sectors (56) and regions (43)

_sectors = ["A01","A02","A03","B","C10-C12","C13-C15","C16","C17","C18","C19","C20","C21","C22","C23","C24","C25","C26","C27","C28","C29","C30","C31_C32","C33","D35","E36","E37-E39","F","G45","G46","G47","H49","H50","H51","H52","H53","I","J58","J59_J60","J61","J62_J63","K64","K65","K66","L68","M69_M70","M71","M72","M73","M74_M75","N","O84","P85","Q","R_S","T","U"]
_regions = ["AUS","AUT","BEL","BGR","BRA","CAN","CHE","CHN","CYP","CZE","DEU","DNK","ESP","EST","FIN","FRA","GBR","GRC","HRV","HUN","IDN","IND","IRL","ITA","JPN","KOR","LTU","LUX","LVA","MEX","MLT","NOR","POL","PRT","ROU", "ROW","RUS","SVK","SVN","SWE","TUR","TWN","USA"]
_Z_multiindex = pd.MultiIndex.from_product(
                [_regions, _sectors], names = [u'region', u'sector'])


In [ ]:
# Create multiindex dataframe
Z_2000 = pd.DataFrame(data=Z_00.values, index=_Z_multiindex, columns=_Z_multiindex)

### **2.1.2. Final Demand (Y)**

* Y(i,i) is final demand satisfied by domestic production - main diagonal.
* Y(i,j) is import to final demand of country j from country i - off diagonal.
* Unit: Millions of USD
* Dimension: (mn x m) : 2408 x 43 in our case.
* Preceeding Steps:
  * Data was first deflated and converted into constant prices where 2010 was the base year. Refer: [Code](https://github.com/GICN/Deflating_WIOD_Tables/blob/master/code/Deflate_tables.R)
  * The long table so obtained needs to be converted back to a matrix form.  Refer: [Code]()

In [ ]:
# Load the data
Y_00 = pd.read_excel("/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/Y_2000.xlsx", header=0, index_col=0)

In [ ]:
# Create multiindex dataframes
_categories = ['final demand']
_fd_multiindex = pd.MultiIndex.from_product(
                 [_regions, _categories], names = [u'region', u'category'])

In [ ]:
Y_2000 = pd.DataFrame(data=Y_00.values, index = _Z_multiindex, columns = _fd_multiindex)

### **2.1.3. Total Value Added (V)**

* Value added at basic prices
* Unit: Millions of USD
* Dimension: (1 x mn) : (1 x 2408) in our case.
* Preceeding Steps:
  * Data was first deflated and converted into constant prices where 2010 was the base year. Refer: [Code](https://github.com/GICN/Deflating_WIOD_Tables/blob/master/code/Deflate_tables.R)
  * The long table so obtained needs to be converted back to a vector form.  Refer: [Code]()

In [ ]:
# Load the data
V_00 = pd.read_excel("/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/V_2000.xlsx", header = 0, index_col=0)

In [ ]:
# Create multiindex dataframes
V_2000 = pd.DataFrame(data=V_00.values, index = ['Value Added'], columns = _Z_multiindex)

### **2.1.4. Total CO2 Emissions (F)**

* F represents total CO2 emissions from production
* Unit: Metric Tonnes (Mt)
* Dimension: (1 x mn) : (1 x 2408) in our case

* Preceeding Steps:
  * Get data for each country, for each sector for 2000 and arrange it in a single row.
  * Done using VBA and Colab





In [ ]:
# Load the data
F_00 = pd.read_excel("/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/F_2000.xlsx", header = 0)

In [ ]:
# Create multiindex dataframes
F_2000 = pd.DataFrame(data=F_00.values, index = ['Emissions'], columns = _Z_multiindex)

## **2.2. Include data in IO System object**

In [ ]:
# Set up an empty IOSystem for 2000
io_00 = pymrio.IOSystem()

In [ ]:
# Add Z and Y values to the system
io_00.Z = Z_2000
io_00.Y = Y_2000

In [ ]:
# We are extending a standard MRIO to an Environmental MRIO so we need to add emissions as an extension
emissions = pymrio.Extension(name = 'Emissions', F=F_2000)
io_00.emissions = emissions
io_00.emissions.unit = pd.DataFrame(data = ['Kt'], index = F_00.index, columns = ['unit'])

In [ ]:
# Adding another extension to explore Value Added in Trade
value_added = pymrio.Extension(name = 'value_added', F=V_2000)
io_00.value_added = value_added
io_00.value_added.unit = pd.DataFrame(data = ['Million USD'], index = V_00.index, columns = ['unit'])

In [ ]:
#Check what is in system
str(io_00)

'IO System with parameters: Z, Y, meta, emissions, value_added'

In [ ]:
# Calculate all missing matrices like x, A, L, S etc
io_00.calc_all()

In [ ]:
# Check what A looks like
io_00.A

region                   AUS                                            \
sector                   A01           A02           A03             B   
region sector                                                            
AUS    A01      1.763960e-01  1.098494e-01  3.761051e-02  5.228905e-04   
       A02      2.225930e-03  2.312480e-02  5.195037e-04  7.258464e-05   
       A03      3.119006e-04  1.033032e-05  4.601233e-03  1.268931e-05   
       B        1.086642e-03  3.644666e-04  3.117794e-04  5.429071e-02   
       C10-C12  2.878419e-02  4.805030e-03  3.101196e-02  1.145171e-03   
...                      ...           ...           ...           ...   
USA    P85      8.947377e-07  2.810061e-07  3.540613e-07  8.769702e-07   
       Q        7.452245e-07  6.174677e-07  3.506312e-07  5.301261e-07   
       R_S      1.707330e-06  1.542567e-06  9.083697e-07  1.189937e-06   
       T        6.708020e-09  4.524688e-09  3.105548e-09  8.307700e-09   
       U        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   

region                                                                  \
sector               C10-C12       C13-C15           C16           C17   
region sector                                                            
AUS    A01      2.593216e-01  6.181741e-02  3.592843e-04  2.762209e-04   
       A02      1.927476e-05  2.307299e-05  1.331732e-01  3.208579e-02   
       A03      6.300529e-03  4.640003e-03  2.220357e-05  2.484000e-05   
       B        2.575509e-03  1.530746e-03  1.913807e-03  3.639975e-03   
       C10-C12  1.460731e-01  4.606915e-02  1.491342e-03  2.250920e-03   
...                      ...           ...           ...           ...   
USA    P85      8.652880e-07  7.343672e-07  6.068690e-07  1.335424e-06   
       Q        3.717461e-07  4.009958e-07  1.088236e-06  6.536527e-07   
       R_S      1.114390e-06  9.243046e-07  2.458351e-06  1.498653e-06   
       T        7.741819e-09  8.723511e-09  1.162765e-08  1.779032e-08   
       U        0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   

region                                      ...           USA                \
sector                   C18           C19  ...           M72           M73   
region sector                               ...                               
AUS    A01      2.103882e-04  9.957240e-05  ...  3.760692e-07  3.759676e-07   
       A02      4.529766e-04  9.357888e-05  ...  1.136458e-07  1.136151e-07   
       A03      2.679160e-05  1.444357e-05  ...  2.318017e-07  2.317390e-07   
       B        1.345666e-03  4.810782e-01  ...  3.477506e-06  3.476567e-06   
       C10-C12  1.696360e-03  5.844862e-04  ...  7.246064e-07  7.244104e-07   
...                      ...           ...  ...           ...           ...   
USA    P85      1.395784e-06  3.675503e-07  ...  2.091667e-03  2.091101e-03   
       Q        7.424741e-07  5.578818e-07  ...  1.042889e-03  1.042607e-03   
       R_S      1.719522e-06  1.494468e-06  ...  9.994060e-03  9.991357e-03   
       T        1.226773e-08  5.604905e-09  ...  2.103157e-04  2.102588e-04   
       U        0.000000e+00  0.000000e+00  ...  0.000000e+00  0.000000e+00   

region                                                                  \
sector               M74_M75             N           O84           P85   
region sector                                                            
AUS    A01      3.754731e-07  5.543237e-07  8.823861e-08  1.151683e-06   
       A02      1.134657e-07  1.655109e-08  3.204390e-07  1.964507e-07   
       A03      2.314343e-07  1.170280e-08  7.094381e-07  4.256307e-07   
       B        3.471994e-06  2.305891e-06  1.883775e-05  1.894565e-06   
       C10-C12  7.234578e-07  2.129164e-07  8.133513e-06  3.635018e-05   
...                      ...           ...           ...           ...   
USA    P85      2.088351e-03  1.315274e-03  2.488937e-03  5.263204e-03   
       Q        1.041235e-03  4.908597e-04  3.614674e-03  2.000939e-04   
     

## **2.3. Aggregating Sectors**

* For the sake of simplicity, we condense 56 sectors into 8 "super-sectors" using the fourth revision of International Standard Industrial Classification.

* The 8 super-sectors are:
  * Sec1: Agriculture, Forestry and Fishing
  * Sec2: Mining and Quarrying
  * Sec3: Manufacturing
  * Sec4: Electricity, Gas and Water Supply
  * Sec5: Construction
  * Sec6: Transportation
  * Sec7: Residential Activities
  * Sec8: Commercial and Public Services

* Reference: [Wang et al 2018](https://doi.org/10.1016/j.apenergy.2018.09.172)



In [ ]:
# Define a dictionary which assigns each of the 56 sectors to one of the 8 super-sectors 
rename_dict = {'A01' : 'Sec1', 'A02' : 'Sec1', 'A03' : 'Sec1', 'B' : 'Sec2', 'C10-C12' : 'Sec3', 'C13-C15' : 'Sec3', 'C16' : 'Sec3', 'C17' : 'Sec3', 'C18' : 'Sec8', 'C19' : 'Sec3', 'C20' : 'Sec3', 'C21' : 'Sec3', 'C22' : 'Sec3', 'C23' : 'Sec3', 'C24' : 'Sec3', 'C25' : 'Sec3', 'C26' : 'Sec3', 'C27' : 'Sec3', 'C28' : 'Sec3', 'C29' : 'Sec3', 'C30' : 'Sec3', 'C31_C32' : 'Sec3', 'C33' : 'Sec8', 'D35' : 'Sec4', 'E36' : 'Sec8', 'E37-E39' : 'Sec8', 'F' : 'Sec5', 'G45' : 'Sec8', 'G46' : 'Sec8', 'G47' : 'Sec8', 'H49' : 'Sec6', 'H50' : 'Sec6', 'H51' : 'Sec6', 'H52' : 'Sec8', 'H53' : 'Sec8', 'I' : 'Sec7', 'J58' : 'Sec8', 'J59_J60' : 'Sec8', 'J61' : 'Sec8', 'J62_J63' : 'Sec8', 'K64' : 'Sec8', 'K65' : 'Sec8', 'K66' : 'Sec8', 'L68' : 'Sec8', 'M69_M70' : 'Sec8', 'M71' : 'Sec8', 'M72' : 'Sec8', 'M73' : 'Sec8', 'M74_M75' : 'Sec8', 'N' : 'Sec8', 'O84' : 'Sec8', 'P85' : 'Sec8', 'Q' : 'Sec8', 'R_S' : 'Sec8', 'T' : 'Sec7', 'U' : 'Sec8'}

In [ ]:
# Rename sectors in the system as per the dictionary 
io_00.rename_sectors(rename_dict)

In [ ]:
# Aggregate sectors with same name
io_00.aggregate_duplicates()

In [ ]:
# Check what Z looks like - should be a mp x mp matrix where p is the number of super-sectors (344 x 344 matrix in our case)
io_00.Z

region                  AUS                                             \
sector                 Sec1          Sec2          Sec3           Sec8   
region sector                                                            
AUS    Sec1     9374.701825     89.997819  20384.862360    2280.934458   
       Sec2       55.917093   8034.086270  31671.820736    3110.168148   
       Sec3     5499.828750  10051.885394  65085.529017   36434.884620   
       Sec8    10879.012006  21540.666784  48924.740862  301514.894123   
       Sec4      256.913367   2333.937541   6250.320470    6398.363206   
...                     ...           ...           ...            ...   
USA    Sec8       36.779803     82.027903    171.420248    1572.798760   
       Sec4        0.054828      0.107364      0.467762       0.382593   
       Sec5        0.002991      0.013219      0.013096       0.111943   
       Sec6       29.153046    101.854149    245.245469    1330.978582   
       Sec7        0.401638      0.732337      3.409654      10.900669   

region                                                                \
sector                Sec4          Sec5          Sec6          Sec7   
region sector                                                          
AUS    Sec1      10.961211    343.769098     35.468812   1566.787279   
       Sec2    2292.617426    535.393808    229.600211    147.384224   
       Sec3    1239.329048  23897.489389   8426.246424  12334.130968   
       Sec8    5392.795511  38939.394552  16432.506986  17882.833635   
       Sec4    7166.252339   1712.909955    164.224881   1076.525130   
...                    ...           ...           ...           ...   
USA    Sec8      18.568905    137.925067     41.720102    133.062431   
       Sec4       0.071304      0.089376      0.073424      0.050220   
       Sec5       0.004592      0.062656      0.003783      0.004319   
       Sec6      32.583216     36.872287     60.077697     31.679322   
       Sec7       0.269180      0.203494      0.469638      0.271139   

region              AUT            ...        TWN                      USA  \
sector             Sec1      Sec2  ...       Sec6       Sec7          Sec1   
region sector                      ...                                       
AUS    Sec1    0.188088  0.001599  ...   0.002506  34.488772     94.419076   
       Sec2    0.009213  0.109238  ...   0.017793   0.013554      7.446371   
       Sec3    0.167797  0.035593  ...   3.540127  65.056831     50.403230   
       Sec8    0.156573  0.152362  ...  13.253130  14.612120     28.468070   
       Sec4    0.001822  0.001783  ...   0.001051   0.008244      0.245852   
...                 ...       ...  ...        ...        ...           ...   
USA    Sec8    4.082703  3.131633  ...  92.701584   6.807989  46217.783729   
       Sec4    0.056960  0.054201  ...   0.068865   0.592502   9439.791794   
       Sec5    0.027889  0.017812  ...   0.000064   0.000098   2024.557835   
       Sec6    0.787469  1.820502  ...  12.627298   4.016012   8552.445602   
       Sec7    0.009536  0.006485  ...   1.239853   0.164889    179.342381   

region                                                                   \
sector                 Sec2           Sec3          Sec8           Sec4   
region sector                                                             
AUS    Sec1        0.235930     230.127606  1.244684e+01       0.062315   
       Sec2      231.441567    1838.306387  9.113911e+01     375.425107   
       Sec3       69.061572    3177.178463  6.352537e+02      19.962592   
       Sec8       51.623895     494.582916  7.163756e+02      21.965332   
       Sec4        1.126198       8.015347  4.867284e+00       1.283723   
...                     ...            ...           ...            ...   
USA    Sec8    73900.886925  750002.370418  3.824398e+06  103557.655105   
       Sec4    16728.265375   90627.879075  1.906222e+05    4286.876130   
       Sec5     4360.696108   17559.

In [ ]:
# Recalculate all matrices like this
io_00.calc_all()

In [ ]:
# Can save all calculated matrices using below code:
io_00.save_all(path='/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Output/io_2000')

# **3. Now, consider multiple years 2000-2014**

* We repeat steps 2.1, 2.2 and 2.3 for all the years using a for-loop and obtain a dictionary of io systems for each year

In [ ]:
# Define the years range
years = range(2000, 2015)

# Define the sector and region lists
sectors = ["A01","A02","A03","B","C10-C12","C13-C15","C16","C17","C18","C19","C20","C21","C22","C23","C24","C25","C26","C27","C28","C29","C30","C31_C32","C33","D35","E36","E37-E39","F","G45","G46","G47","H49","H50","H51","H52","H53","I","J58","J59_J60","J61","J62_J63","K64","K65","K66","L68","M69_M70","M71","M72","M73","M74_M75","N","O84","P85","Q","R_S","T","U"]
regions = ["AUS","AUT","BEL","BGR","BRA","CAN","CHE","CHN","CYP","CZE","DEU","DNK","ESP","EST","FIN","FRA","GBR","GRC","HRV","HUN","IDN","IND","IRL","ITA","JPN","KOR","LTU","LUX","LVA","MEX","MLT","NOR","POL","PRT","ROU", "ROW", "RUS","SVK","SVN","SWE","TUR","TWN", "USA"]

# Create a dictionary to store the io objects for each year
io_dict = {}

# Iterate over the years
for year in years:
    print(f"Working on {year}")
    # Read the Z file for the current year
    z_file = f"/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/Z_{year}.xlsx"
    z_data = pd.read_excel(z_file, header=0, index_col=0)
    
    # Create the Z DataFrame with the multi-index
    z_multiindex = pd.MultiIndex.from_product([regions, sectors], names=["region", "sector"])
    z_df = pd.DataFrame(data=z_data.values, index=z_multiindex, columns=z_multiindex)
    
    # Read the Y file for the current year
    y_file = f"/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/Y_{year}.xlsx"
    y_data = pd.read_excel(y_file, header=0, index_col=0)
    
    # Create the Y DataFrame with the multi-index
    y_multiindex = pd.MultiIndex.from_product([regions, ["final demand"]], names=["region", "category"])
    y_df = pd.DataFrame(data=y_data.values, index=z_multiindex, columns=y_multiindex)
    
    # Read the F file for the current year
    f_file = f"/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/F_{year}.xlsx"
    f_data = pd.read_excel(f_file, header=0)
    
    # Create the F DataFrame
    f_df = pd.DataFrame(data=f_data.values, index=["Emissions"], columns=z_multiindex)
    
    # Read the V file for the current year
    v_file = f"/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Input/V_{year}.xlsx"
    v_data = pd.read_excel(v_file, header=0, index_col=0)
    
    # Create the V DataFrame
    v_df = pd.DataFrame(data=v_data.values, index=["Value Added"], columns=z_multiindex)
    
    # Create the IOSystem for the current year
    io = pymrio.IOSystem()
    io.Z = z_df
    io.Y = y_df
    io.emissions = pymrio.Extension(name="Emissions", F=f_df)
    io.emissions.unit = pd.DataFrame(data=["Kt"], index=f_data.index, columns=["unit"])
    io.value_added = pymrio.Extension(name="value_added", F=v_df)
    io.value_added.unit = pd.DataFrame(data=["Million USD"], index=v_data.index, columns=["unit"])
    
    # Calculate all matrices
    io.calc_all()
    
    # Define the rename dictionary
    rename_dict = {'A01': 'Sec1', 'A02': 'Sec1', 'A03': 'Sec1', 'B': 'Sec2', 'C10-C12': 'Sec3', 'C13-C15': 'Sec3',
                   'C16': 'Sec3', 'C17': 'Sec3', 'C18': 'Sec8', 'C19': 'Sec3', 'C20': 'Sec3', 'C21': 'Sec3',
                   'C22': 'Sec3', 'C23': 'Sec3', 'C24': 'Sec3', 'C25': 'Sec3', 'C26': 'Sec3', 'C27': 'Sec3',
                   'C28': 'Sec3', 'C29': 'Sec3', 'C30': 'Sec3', 'C31_C32': 'Sec3', 'C33': 'Sec8', 'D35': 'Sec4',
                   'E36': 'Sec8', 'E37-E39': 'Sec8', 'F': 'Sec5', 'G45': 'Sec8', 'G46': 'Sec8', 'G47': 'Sec8',
                   'H49': 'Sec6', 'H50': 'Sec6', 'H51': 'Sec6', 'H52': 'Sec8', 'H53': 'Sec8', 'I': 'Sec7',
                   'J58': 'Sec8', 'J59_J60': 'Sec8', 'J61': 'Sec8', 'J62_J63': 'Sec8', 'K64': 'Sec8', 'K65': 'Sec8',
                   'K66': 'Sec8', 'L68': 'Sec8', 'M69_M70': 'Sec8', 'M71': 'Sec8', 'M72': 'Sec8', 'M73': 'Sec8',
                   'M74_M75': 'Sec8', 'N': 'Sec8', 'O84': 'Sec8', 'P85': 'Sec8', 'Q': 'Sec8', 'R_S': 'Sec8',
                   'T': 'Sec7', 'U': 'Sec8'}
    
    # Rename sectors and aggregate duplicates
    io.rename_sectors(rename_dict)
    io.aggregate_duplicates()
    
    # Recalculate all matrices
    io.calc_all()

    # Save the IOSystem data to files
    output_path = f"/content/drive/MyDrive/Masters Thesis - Yashna Kumar/Code/Output/io_{year}"
    io.save_all(path=output_path)

    # Store the io object in the dictionary
    io_dict[year] = io


Working on 2000
Working on 2001
Working on 2002
Working on 2003
Working on 2004
Working on 2005
Working on 2006
Working on 2007
Working on 2008
Working on 2009
Working on 2010
Working on 2011
Working on 2012
Working on 2013
Working on 2014


# **4. Structural Decomposition Analysis**

* A structural decomposition analysis can be done in multiple ways. We do a basic SDA as explained by *Miller and Blair (2009)* with a slight modification inspired by *Kim and Tromp (2021)*. The exact formula is mentioned in our paper. 

* We first consider the entire timeline and analys the drivers of changes in emissions between 2000 and 2014. Later, we break the time period into 3 parts: 2000-2008, 2008-2011, 2011-14.

## **4.1. Between 2000 and 2014**

### **4.1.1. Calculating Required Matrices**

* To proceed, we first need to calculate various matrices like del_S, del_L etc (refer the SDA formula).

In [ ]:
# Change in CO2 emission intensity (S) by each country, in each sector over the period 2000 - 2014 : (1 x 344) matrix
del_S = io_dict[2014].emissions.S - io_dict[2000].emissions.S

In [ ]:
# Change in final demand (Y) by each country satisfied by domestic production or imports over the period 2000 - 2014 : (344 x 43) matrix
del_Y = io_dict[2014].Y - io_dict[2000].Y

In [ ]:
# Change in Total Requirements or Leontief matrix (L) over the period 2000 - 2014 : (344 x 344) matrix
del_L = io_dict[2014].L - io_dict[2000].L

In [ ]:
# Change in input requirement (A) for each sector, for each country over the period 2000 - 2014 : (344 x 344) matrix
del_A = io_dict[2014].A - io_dict[2000].A

In [ ]:
# del_A_11 = Change in A_11, that is, change in structure of domestic intermediate inputs in India 
# Same as del_A but all values = 0 except when row region index = column region index = 'IND' 

# Extract the row and column indices for Region IND
indices = del_A.index.get_level_values("region") == "IND"

# Create a new dataframe which has the same format as del_A but all values = 0
del_A_11 = pd.DataFrame(0, index=del_A.index, columns=del_A.columns)

# Assign the values from del_A where both row and column indices are "IND"
del_A_11.loc[indices, indices] = del_A.loc[indices, indices]
del_A_11

region         AUS                                     AUT       ...  TWN  \
sector        Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7 Sec1 Sec2  ... Sec6   
region sector                                                    ...        
AUS    Sec1      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec2      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec3      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec8      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec4      0    0    0    0    0    0    0    0    0    0  ...    0   
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
USA    Sec8      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec4      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec5      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec6      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec7      0    0    0    0    0    0    0    0    0    0  ...    0   

region              USA                                     
sector        Sec7 Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7  
region sector                                               
AUS    Sec1      0    0    0    0    0    0    0    0    0  
       Sec2      0    0    0    0    0    0    0    0    0  
       Sec3      0    0    0    0    0    0    0    0    0  
       Sec8      0    0    0    0    0    0    0    0    0  
       Sec4      0    0    0    0    0    0    0    0    0  
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  
USA    Sec8      0    0    0    0    0    0    0    0    0  
       Sec4      0    0    0    0    0    0    0    0    0  
       Sec5      0    0    0    0    0    0    0    0    0  
       Sec6      0    0    0    0    0    0    0    0    0  
       Sec7      0    0    0    0    0    0    0    0    0  

[344 rows x 344 columns]

In [ ]:
# del_A_1_star = Change in A_1_star, that is, change in structure of intermediate inputs in India's exports
# Same as del_A but all values = 0 except when row region index = 'IND' and column region index ≠ 'IND'

# Filter the rows where the row region is "IND"
ind_rows = del_A.index.get_level_values("region") == "IND"

# Filter the columns where the column region is not "IND"
other_cols = del_A.columns.get_level_values("region") != "IND"

# Create a new dataframe which has the same format as del_A but all values = 0
del_A_1_star = pd.DataFrame(0, index=del_A.index, columns=del_A.columns)

# Assign the values from del_A where row region is "IND" and column region is not "IND"
del_A_1_star.loc[ind_rows, other_cols] = del_A.loc[ind_rows, other_cols]
del_A_1_star

region         AUS                                     AUT       ...  TWN  \
sector        Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7 Sec1 Sec2  ... Sec6   
region sector                                                    ...        
AUS    Sec1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
USA    Sec8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec5    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec6    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec7    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

region              USA                                     
sector        Sec7 Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7  
region sector                                               
AUS    Sec1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  
USA    Sec8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec5    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec6    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec7    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[344 rows x 344 columns]

In [ ]:
# del_A_star_ = Change in A_star_1, that is, change in structure of intermediate inputs in India's imports
# Same as del_A but all values = 0 except when row region index ≠ 'IND' and column region index = 'IND'

# Filter the rows where the row region is not "IND"
other_rows = del_A.index.get_level_values("region") != "IND"

# Filter the columns where the column region is "IND"
ind_cols = del_A.columns.get_level_values("region") == "IND"

# Create a new dataframe which has the same format as del_A but all values = 0
del_A_star_1 = pd.DataFrame(0, index=del_A.index, columns=del_A.columns)

# Assign the values from del_A where row region is not "IND" and column region is "IND"
del_A_star_1.loc[other_rows, ind_cols] = del_A.loc[other_rows, ind_cols]
del_A_star_1

region         AUS                                     AUT       ...  TWN  \
sector        Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7 Sec1 Sec2  ... Sec6   
region sector                                                    ...        
AUS    Sec1      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec2      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec3      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec8      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec4      0    0    0    0    0    0    0    0    0    0  ...    0   
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
USA    Sec8      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec4      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec5      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec6      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec7      0    0    0    0    0    0    0    0    0    0  ...    0   

region              USA                                     
sector        Sec7 Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7  
region sector                                               
AUS    Sec1      0    0    0    0    0    0    0    0    0  
       Sec2      0    0    0    0    0    0    0    0    0  
       Sec3      0    0    0    0    0    0    0    0    0  
       Sec8      0    0    0    0    0    0    0    0    0  
       Sec4      0    0    0    0    0    0    0    0    0  
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  
USA    Sec8      0    0    0    0    0    0    0    0    0  
       Sec4      0    0    0    0    0    0    0    0    0  
       Sec5      0    0    0    0    0    0    0    0    0  
       Sec6      0    0    0    0    0    0    0    0    0  
       Sec7      0    0    0    0    0    0    0    0    0  

[344 rows x 344 columns]

In [ ]:
# del_A_star_star = Change in A_star_star, that is, change in structure of intermediate inputs in countries other than India
# Same as del_A but all values = 0 except when row region index ≠ 'IND' and column region index ≠ 'IND'

# Filter the rows where the row region is not "IND"
other_rows = del_A.index.get_level_values("region") != "IND"

# Filter the columns where the column region is "IND"
other_cols = del_A.columns.get_level_values("region") != "IND"

# Create a new dataframe which has the same format as del_A but all values = 0
del_A_star_star = pd.DataFrame(0, index=del_A.index, columns=del_A.columns)

# Assign the values from del_A where row region is not "IND" and column region is "IND"
del_A_star_star.loc[other_rows, other_cols] = del_A.loc[other_rows, other_cols]
del_A_star_star

region                  AUS                                            \
sector                 Sec1          Sec2          Sec3          Sec8   
region sector                                                           
AUS    Sec1   -4.358022e-03  2.197314e-03  1.804438e-02  5.483009e-05   
       Sec2    2.110417e-03 -7.279110e-03  7.937699e-03  1.384862e-03   
       Sec3   -2.583839e-02 -1.611007e-02 -7.165316e-02 -1.301517e-02   
       Sec8   -4.662661e-03  2.992028e-02 -1.137574e-02 -1.416548e-02   
       Sec4    1.757232e-03 -2.923161e-04 -7.157949e-04 -5.603157e-04   
...                     ...           ...           ...           ...   
USA    Sec8    2.035058e-04  4.940388e-04  1.457518e-04  1.938932e-04   
       Sec4   -8.925935e-07 -6.241519e-07 -1.331382e-06 -2.440194e-07   
       Sec5    1.741836e-06  3.428195e-06  1.629797e-07  1.225318e-06   
       Sec6   -3.299478e-04 -4.090242e-04 -4.664683e-04 -7.898897e-04   
       Sec7   -5.939822e-06 -7.452381e-07 -8.720518e-06 -8.593593e-06   

region                                                                  AUT  \
sector             Sec4          Sec5      Sec6          Sec7          Sec1   
region sector                                                                 
AUS    Sec1   -0.000025 -7.701369e-05  0.000105 -7.173790e-03 -1.697597e-05   
       Sec2   -0.027857  8.564530e-03  0.001533  1.925795e-03  2.669014e-06   
       Sec3   -0.011134 -2.121417e-02 -0.055808 -3.244869e-02  4.126643e-05   
       Sec8    0.036752 -6.090795e-02  0.012371 -6.036555e-02  9.013454e-06   
       Sec4    0.092254 -7.098305e-03  0.001082 -1.606031e-03  6.450720e-07   
...                 ...           ...       ...           ...           ...   
USA    Sec8    0.000537  2.346027e-04  0.000479 -9.229158e-04  4.153342e-05   
       Sec4   -0.000001 -3.873041e-07 -0.000001 -6.449306e-07 -7.373139e-07   
       Sec5    0.000004  2.497355e-05  0.000001  6.721952e-07 -2.525411e-06   
       Sec6   -0.000565 -9.384942e-05 -0.000415 -3.599751e-04 -3.604430e-05   
       Sec7   -0.000006  1.549452e-06 -0.000005 -2.959855e-06 -5.281392e-07   

region                       ...           TWN                         USA  \
sector                 Sec2  ...          Sec6          Sec7          Sec1   
region sector                ...                                             
AUS    Sec1   -5.793863e-07  ...  1.657293e-07 -1.987448e-03 -2.283589e-04   
       Sec2    1.383334e-04  ...  3.729171e-05  3.235569e-06 -1.886680e-05   
       Sec3    3.102479e-06  ...  5.313402e-04 -3.030185e-03 -3.905742e-05   
       Sec8   -1.363229e-05  ... -4.693285e-04 -7.268689e-04 -3.814477e-05   
       Sec4    1.634736e-06  ...  1.409522e-07 -1.777372e-07 -4.305046e-07   
...                     ...  ...           ...           ...           ...   
USA    Sec8    4.377281e-04  ...  1.237406e-02 -2.235196e-04 -5.237469e-03   
       Sec4   -8.484815e-06  ... -2.942128e-06 -3.049352e-05 -1.966179e-02   
       Sec5   -7.142059e-06  ... -3.304195e-09 -4.629097e-09 -3.658066e-04   
       Sec6   -3.083822e-04  ... -1.061729e-04 -1.211688e-04  1.706540e-04   
       Sec7   -1.820817e-06  ... -6.219407e-05 -7.974027e-06  3.989769e-04   

region                                                             \
sector                 Sec2      Sec3          Sec8          Sec4   
region sector                                                       
AUS    Sec1   -4.240472e-07 -0.000035 -6.622072e-07 -8.518646e-08   
       Sec2   -5.102506e-04 -0.000318 -4.919788e-06 -6.608778e-04   
       Sec3   -1.108115e-04 -0.000278 -9.334067e-06 -2.851657e-05   
       Sec8   -6.976880e-05  0.000013 -1.910319e-06 -2.416883e-05   
       Sec4   -2.467354e-06 -0.000001 -1.163889e-07 -2.217600e-06   
...                     ...       ...           ...           ...   
USA    Sec8   -6.494323e-02  0.008607  1.282589e-02 -8.460796e-02   
       Sec4   -3.452882e-02 -0.009339 -6.890791e-03 -2.534236e-03   
       Sec5   -2.7

### **4.1.2. Miller and Blair SDA**

* Initially, decomposing emissions into intensity, technology and demand effect as per formula in [Miller and Blair, 2009](http://digamo.free.fr/io2009.pdf)




In [ ]:
intensity_effect = (1/2) * (del_S @ ((io_dict[2000].L @ io_dict[2000].Y) + (io_dict[2014].L @ io_dict[2014].Y)))
intensity_effect

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,-116.293351,-35.18693,-42.54126,-13.086278,-46.625246,-120.09402,-32.64868,-3780.356659,-4.106266,-41.294584,...,-27.640824,-47.042197,-2326.546524,-199.087903,-16.867087,-6.656384,-33.628287,-73.440137,-134.27059,-661.548977


In [ ]:
#Isolating effect for India
intensity_effect.loc[:, 'IND']

category,final demand
Emissions,-210.923691


In [ ]:
tech_effect = (1/2) * ((io_dict[2000].emissions.S @ del_L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ del_L @ io_dict[2000].Y))
tech_effect

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,52.602782,12.147382,25.160413,-6.536638,73.535913,41.66002,17.84195,1723.633976,1.656524,-4.02773,...,10.171781,-6.150608,1890.052675,-52.030747,-2.347909,2.005004,11.340897,8.147879,57.293957,-542.100505


In [ ]:
#Isolating effect for India
tech_effect.loc[:, 'IND']

category,final demand
Emissions,-103.29658


In [ ]:
dd_effect = (1/2) * (((io_dict[2000].emissions.S @ io_dict[2000].L) + (io_dict[2014].emissions.S @ io_dict[2014].L)) @ del_Y)
dd_effect

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,187.871833,11.702825,5.427311,3.931643,214.510415,150.964494,24.907181,7377.995903,3.954376,18.73152,...,-8.459752,55.08442,3536.102,597.930346,9.84343,0.555251,23.707218,133.90189,-20.51191,997.766657


In [ ]:
#Isolating effect for India
dd_effect.loc[:, 'IND']

category,final demand
Emissions,1368.872905


In [ ]:
tot_eff_ind = intensity_effect.loc[:, 'IND'] + tech_effect.loc[:, 'IND'] + dd_effect.loc[:, 'IND']
tot_eff_ind

category,final demand
Emissions,1054.652634


### **4.1.3. Further decompsing the technology effect**

* Decomposing del_L into:
  1. change in structure of domestic intermediate inputs in India 
  2. change in structure of intermediate inputs in India’s exports
  3. change in structure of intermediate inputs in India’s imports
  4. change in the structure of intermediate inputs in countries other than India

In [ ]:
# change in structure of domestic intermediate inputs in India
effect_A_11 = (1/2) * (((io_00.emissions.S @ io_dict[2014].L @ del_A_11 @ io_dict[2000].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_11 @ io_dict[2000].L @ io_dict[2000].Y)))
effect_A_11

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,-0.324892,-0.090093,-0.245503,-0.008767,-0.401144,-0.495476,-0.163209,-1.163101,-0.013578,-0.032275,...,-0.080588,-0.021278,-15.388406,-0.541757,-0.019251,-0.013179,-0.126145,-0.407206,-0.195669,-4.630498


In [ ]:
effect_A_11.loc[:,'IND']

category,final demand
Emissions,-203.263159


In [ ]:
# change in structure of intermediate inputs in India’s exports
effect_A_1_star = (1/2) * (((io_dict[2000].emissions.S @ io_dict[2014].L @ del_A_1_star @ io_dict[2000].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_1_star @ io_dict[2000].L @ io_dict[2000].Y)))
effect_A_1_star

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,1.707587,0.358324,0.971038,0.179823,6.478485,1.692022,0.5723,3.095867,0.034235,0.298601,...,0.495036,0.23106,33.731937,0.932439,0.099669,0.09347,0.549305,5.342541,1.216995,23.185521


In [ ]:
effect_A_1_star.loc[:,'IND']

category,final demand
Emissions,0.604341


In [ ]:
# change in structure of intermediate inputs in India’s imports
effect_A_star_1 = (1/2) * (((io_dict[2000].emissions.S @ io_dict[2014].L @ del_A_star_1 @ io_dict[2000].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_star_1 @ io_dict[2000].L @ io_dict[2000].Y)))
effect_A_star_1

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,0.107811,0.028703,0.070109,0.002664,0.126697,0.151216,0.048344,0.327477,0.003238,0.009415,...,0.023274,0.006856,4.65583,0.147007,0.006815,0.004144,0.033526,0.141386,0.057148,1.439408


In [ ]:
effect_A_star_1.loc[:,'IND']

category,final demand
Emissions,58.585269


In [ ]:
# change in the structure of intermediate inputs in countries other than India. 
effect_A_star_star = (1/2) * (((io_dict[2000].emissions.S @ io_dict[2014].L @ del_A_star_star @ io_dict[2000].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_star_star @ io_dict[2000].L @ io_dict[2000].Y)))
effect_A_star_star

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,51.112276,11.850449,24.36477,-6.710359,67.331875,40.312257,17.384515,1721.373733,1.63263,-4.303471,...,9.734059,-6.367247,1867.053314,-52.568436,-2.435142,1.920568,10.88421,3.071158,56.215483,-562.094936


In [ ]:
effect_A_star_star.loc[:,'IND']

category,final demand
Emissions,40.776969


## **Can also do this for different/smaller sections of the time period:**

## **4.2. Between 2005 and 2014**

### **4.2.1. Calculating Required Matrices**

* To proceed, we first need to calculate various matrices like del_S, del_L etc (refer the SDA formula).

In [ ]:
# Change in CO2 emission intensity (S) by each country, in each sector over the period 2000 - 2014 : (1 x 344) matrix
del_S = io_dict[2014].emissions.S - io_dict[2005].emissions.S

In [ ]:
# Change in final demand (Y) by each country satisfied by domestic production or imports over the period 2000 - 2014 : (344 x 43) matrix
del_Y = io_dict[2014].Y - io_dict[2005].Y

In [ ]:
# Change in Total Requirements or Leontief matrix (L) over the period 2000 - 2014 : (344 x 344) matrix
del_L = io_dict[2014].L - io_dict[2005].L

In [ ]:
# Change in input requirement (A) for each sector, for each country over the period 2000 - 2014 : (344 x 344) matrix
del_A = io_dict[2014].A - io_dict[2005].A

In [ ]:
# del_A_11 = Change in A_11, that is, change in structure of domestic intermediate inputs in India 
# Same as del_A but all values = 0 except when row region index = column region index = 'IND' 

# Extract the row and column indices for Region IND
indices = del_A.index.get_level_values("region") == "IND"

# Create a new dataframe which has the same format as del_A but all values = 0
del_A_11 = pd.DataFrame(0, index=del_A.index, columns=del_A.columns)

# Assign the values from del_A where both row and column indices are "IND"
del_A_11.loc[indices, indices] = del_A.loc[indices, indices]
del_A_11

region         AUS                                     AUT       ...  TWN  \
sector        Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7 Sec1 Sec2  ... Sec6   
region sector                                                    ...        
AUS    Sec1      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec2      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec3      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec8      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec4      0    0    0    0    0    0    0    0    0    0  ...    0   
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
USA    Sec8      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec4      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec5      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec6      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec7      0    0    0    0    0    0    0    0    0    0  ...    0   

region              USA                                     
sector        Sec7 Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7  
region sector                                               
AUS    Sec1      0    0    0    0    0    0    0    0    0  
       Sec2      0    0    0    0    0    0    0    0    0  
       Sec3      0    0    0    0    0    0    0    0    0  
       Sec8      0    0    0    0    0    0    0    0    0  
       Sec4      0    0    0    0    0    0    0    0    0  
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  
USA    Sec8      0    0    0    0    0    0    0    0    0  
       Sec4      0    0    0    0    0    0    0    0    0  
       Sec5      0    0    0    0    0    0    0    0    0  
       Sec6      0    0    0    0    0    0    0    0    0  
       Sec7      0    0    0    0    0    0    0    0    0  

[344 rows x 344 columns]

In [ ]:
# del_A_1_star = Change in A_1_star, that is, change in structure of intermediate inputs in India's exports
# Same as del_A but all values = 0 except when row region index = 'IND' and column region index ≠ 'IND'

# Filter the rows where the row region is "IND"
ind_rows = del_A.index.get_level_values("region") == "IND"

# Filter the columns where the column region is not "IND"
other_cols = del_A.columns.get_level_values("region") != "IND"

# Create a new dataframe which has the same format as del_A but all values = 0
del_A_1_star = pd.DataFrame(0, index=del_A.index, columns=del_A.columns)

# Assign the values from del_A where row region is "IND" and column region is not "IND"
del_A_1_star.loc[ind_rows, other_cols] = del_A.loc[ind_rows, other_cols]
del_A_1_star

region         AUS                                     AUT       ...  TWN  \
sector        Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7 Sec1 Sec2  ... Sec6   
region sector                                                    ...        
AUS    Sec1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
USA    Sec8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec5    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec6    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       Sec7    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

region              USA                                     
sector        Sec7 Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7  
region sector                                               
AUS    Sec1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  
USA    Sec8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec5    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec6    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       Sec7    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[344 rows x 344 columns]

In [ ]:
# del_A_star_ = Change in A_star_1, that is, change in structure of intermediate inputs in India's imports
# Same as del_A but all values = 0 except when row region index ≠ 'IND' and column region index = 'IND'

# Filter the rows where the row region is not "IND"
other_rows = del_A.index.get_level_values("region") != "IND"

# Filter the columns where the column region is "IND"
ind_cols = del_A.columns.get_level_values("region") == "IND"

# Create a new dataframe which has the same format as del_A but all values = 0
del_A_star_1 = pd.DataFrame(0, index=del_A.index, columns=del_A.columns)

# Assign the values from del_A where row region is not "IND" and column region is "IND"
del_A_star_1.loc[other_rows, ind_cols] = del_A.loc[other_rows, ind_cols]
del_A_star_1

region         AUS                                     AUT       ...  TWN  \
sector        Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7 Sec1 Sec2  ... Sec6   
region sector                                                    ...        
AUS    Sec1      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec2      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec3      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec8      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec4      0    0    0    0    0    0    0    0    0    0  ...    0   
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
USA    Sec8      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec4      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec5      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec6      0    0    0    0    0    0    0    0    0    0  ...    0   
       Sec7      0    0    0    0    0    0    0    0    0    0  ...    0   

region              USA                                     
sector        Sec7 Sec1 Sec2 Sec3 Sec8 Sec4 Sec5 Sec6 Sec7  
region sector                                               
AUS    Sec1      0    0    0    0    0    0    0    0    0  
       Sec2      0    0    0    0    0    0    0    0    0  
       Sec3      0    0    0    0    0    0    0    0    0  
       Sec8      0    0    0    0    0    0    0    0    0  
       Sec4      0    0    0    0    0    0    0    0    0  
...            ...  ...  ...  ...  ...  ...  ...  ...  ...  
USA    Sec8      0    0    0    0    0    0    0    0    0  
       Sec4      0    0    0    0    0    0    0    0    0  
       Sec5      0    0    0    0    0    0    0    0    0  
       Sec6      0    0    0    0    0    0    0    0    0  
       Sec7      0    0    0    0    0    0    0    0    0  

[344 rows x 344 columns]

In [ ]:
# del_A_star_star = Change in A_star_star, that is, change in structure of intermediate inputs in countries other than India
# Same as del_A but all values = 0 except when row region index ≠ 'IND' and column region index ≠ 'IND'

# Filter the rows where the row region is not "IND"
other_rows = del_A.index.get_level_values("region") != "IND"

# Filter the columns where the column region is "IND"
other_cols = del_A.columns.get_level_values("region") != "IND"

# Create a new dataframe which has the same format as del_A but all values = 0
del_A_star_star = pd.DataFrame(0, index=del_A.index, columns=del_A.columns)

# Assign the values from del_A where row region is not "IND" and column region is "IND"
del_A_star_star.loc[other_rows, other_cols] = del_A.loc[other_rows, other_cols]
del_A_star_star

region                  AUS                                            \
sector                 Sec1          Sec2          Sec3          Sec8   
region sector                                                           
AUS    Sec1   -1.509225e-02  2.255454e-03  1.285866e-02 -9.554922e-05   
       Sec2    1.867122e-03 -1.445784e-02  6.419810e-03  5.784961e-04   
       Sec3   -1.970409e-02 -9.460285e-03 -6.400517e-02 -1.111509e-02   
       Sec8    5.624639e-03  4.141689e-02 -2.044811e-03 -1.117794e-02   
       Sec4    2.315023e-03  1.700286e-03  2.288945e-03  1.005499e-05   
...                     ...           ...           ...           ...   
USA    Sec8    3.580882e-04  5.961161e-04  2.103192e-04  3.139712e-04   
       Sec4   -6.464393e-08 -1.017561e-07 -1.406971e-07 -1.090105e-09   
       Sec5    1.744149e-06  3.394941e-06  1.776560e-07  1.257732e-06   
       Sec6   -5.533184e-05 -4.630546e-05 -4.709128e-05 -1.014613e-04   
       Sec7   -4.835463e-06  9.612806e-08 -6.853179e-06 -7.179237e-06   

region                                                                 \
sector                 Sec4          Sec5          Sec6          Sec7   
region sector                                                           
AUS    Sec1    4.056294e-05  1.396822e-05  1.743232e-04 -1.022957e-02   
       Sec2   -3.777622e-02  7.849623e-03  8.067698e-04  1.358114e-03   
       Sec3   -7.083826e-03 -7.306570e-03 -4.782845e-02 -2.546664e-02   
       Sec8    4.078345e-02 -5.090714e-02  2.532171e-02 -5.758341e-02   
       Sec4    1.088639e-01 -4.757903e-03  1.427212e-03 -7.025525e-04   
...                     ...           ...           ...           ...   
USA    Sec8    5.459873e-04  4.112029e-04  6.381868e-04 -7.823778e-04   
       Sec4   -8.731895e-07  4.513470e-08  6.686508e-08 -1.500123e-07   
       Sec5    4.020882e-06  2.456992e-05  1.396910e-06  6.741733e-07   
       Sec6   -1.200342e-04  1.483354e-05  1.150561e-04 -1.323462e-04   
       Sec7   -5.096568e-06  1.734404e-06 -4.169862e-06 -2.801260e-06   

region                  AUT                ...           TWN                \
sector                 Sec1          Sec2  ...          Sec6          Sec7   
region sector                              ...                               
AUS    Sec1   -4.385701e-06 -7.540710e-08  ... -5.033107e-09 -7.151052e-04   
       Sec2    2.978078e-06  1.649941e-04  ...  3.500174e-05  3.145148e-06   
       Sec3    2.907869e-05  2.412647e-06  ... -3.140732e-04 -1.314659e-03   
       Sec8    7.330504e-06 -1.676122e-06  ... -5.660215e-04 -3.077172e-04   
       Sec4    7.286929e-07  2.113117e-06  ...  1.378376e-07  1.709077e-07   
...                     ...           ...  ...           ...           ...   
USA    Sec8    9.535088e-05  9.617069e-04  ...  9.024216e-03 -4.736152e-05   
       Sec4   -4.765892e-09  1.188511e-06  ... -1.649144e-06 -1.279032e-05   
       Sec5   -1.254914e-06 -2.373911e-06  ... -3.004462e-09 -2.308933e-09   
       Sec6   -1.880595e-05  2.700385e-05  ... -6.152029e-05  1.373844e-05   
       Sec7   -7.041101e-07 -1.419940e-06  ... -8.533231e-05 -4.188210e-06   

region                  USA                                            \
sector                 Sec1          Sec2          Sec3          Sec8   
region sector                                                           
AUS    Sec1   -5.274757e-05 -1.163191e-07 -9.209865e-06 -5.695985e-08   
       Sec2   -3.508332e-06 -1.631573e-04 -7.873723e-05 -1.437397e-06   
       Sec3    2.075627e-05 -3.351209e-05 -4.646849e-05  5.873208e-06   
       Sec8   -7.291613e-06 -7.255508e-06  3.940094e-05 -6.180864e-06   
       Sec4   -8.331057e-09 -4.798347e-07 -1.495984e-07  1.989825e-08   
...                     ...           ...           ...           ...   
USA    Sec8    1.076981e-02 -1.365474e-02  4.151003e-03  4.304440e-03   
       Sec4   -5.932004e-03 -7.848975e-03 -4.092321e-03 -2.933295e-03   
       Sec5   -2.097011e-04 -7.240083e-03  3.663637e-04 -1

### **4.2.2. Miller and Blair SDA**

* Initially, decomposing emissions into intensity, technology and demand effect as per formula in [Miller and Blair, 2009](http://digamo.free.fr/io2009.pdf)




In [ ]:
intensity_effect = (1/2) * (del_S @ ((io_dict[2005].L @ io_dict[2005].Y) + (io_dict[2014].L @ io_dict[2014].Y)))
intensity_effect

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,-94.622147,-27.970663,-28.369749,-7.237364,6.061634,-79.671728,-20.013416,-2577.673508,-2.680516,-23.291812,...,-22.817332,-34.922229,-1115.969021,-48.447124,-13.158461,-4.334417,-22.222889,-40.534154,-80.20062,-593.628466


In [ ]:
#Isolating effect for India
intensity_effect.loc[:, 'IND']

category,final demand
Emissions,-86.327235


In [ ]:
tech_effect = (1/2) * ((io_dict[2005].emissions.S @ del_L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ del_L @ io_dict[2005].Y))
tech_effect

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,35.821176,2.677587,15.90929,-5.986039,44.572061,22.763972,8.913638,353.9243,1.531318,-2.742979,...,3.660997,-0.576534,560.513419,-125.477338,-1.027169,1.009944,3.991151,6.609353,24.426516,-291.928072


In [ ]:
#Isolating effect for India
tech_effect.loc[:, 'IND']

category,final demand
Emissions,-67.182524


In [ ]:
dd_effect = (1/2) * (((io_dict[2005].emissions.S @ io_dict[2005].L) + (io_dict[2014].emissions.S @ io_dict[2014].L)) @ del_Y)
dd_effect

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,102.562762,5.038202,2.213258,-0.242487,207.871422,98.680124,22.391839,5951.616848,-0.693735,4.543646,...,-6.897755,24.533863,2453.774537,309.257525,3.177279,-0.490756,17.441557,98.221724,11.660044,378.202238


In [ ]:
#Isolating effect for India
dd_effect.loc[:, 'IND']

category,final demand
Emissions,1006.531009


In [ ]:
tot_eff_ind = intensity_effect.loc[:, 'IND'] + tech_effect.loc[:, 'IND'] + dd_effect.loc[:, 'IND']
tot_eff_ind

category,final demand
Emissions,853.02125


### **4.2.3. Further decompsing the technology effect**

* Decomposing del_L into:
  1. change in structure of domestic intermediate inputs in India 
  2. change in structure of intermediate inputs in India’s exports
  3. change in structure of intermediate inputs in India’s imports
  4. change in the structure of intermediate inputs in countries other than India

In [ ]:
# change in structure of domestic intermediate inputs in India
effect_A_11 = (1/2) * (((io_dict[2005].emissions.S @ io_dict[2014].L @ del_A_11 @ io_dict[2005].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_11 @ io_dict[2005].L @ io_dict[2005].Y)))
effect_A_11

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,-0.21497,-0.052541,-0.142058,-0.007128,-0.36333,-0.279771,-0.076248,-1.564046,-0.007468,-0.023407,...,-0.038986,-0.0216,-9.785017,-0.171465,-0.015806,-0.011604,-0.078989,-0.259195,-0.105546,-3.012865


In [ ]:
effect_A_11.loc[:,'IND']

category,final demand
Emissions,-90.014823


In [ ]:
# change in structure of intermediate inputs in India’s exports
effect_A_1_star = (1/2) * (((io_dict[2005].emissions.S @ io_dict[2014].L @ del_A_1_star @ io_dict[2005].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_1_star @ io_dict[2005].L @ io_dict[2005].Y)))
effect_A_1_star

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,1.054672,0.235897,0.425085,0.157616,3.974956,0.925897,0.514185,-36.032078,0.02525,0.186579,...,0.415217,0.011898,10.928644,0.388793,0.071109,0.056762,0.2939,4.373009,0.595163,13.883776


In [ ]:
effect_A_1_star.loc[:,'IND']

category,final demand
Emissions,0.131209


In [ ]:
# change in structure of intermediate inputs in India’s imports
effect_A_star_1 = (1/2) * (((io_dict[2005].emissions.S @ io_dict[2014].L @ del_A_star_1 @ io_dict[2005].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_star_1 @ io_dict[2005].L @ io_dict[2005].Y)))
effect_A_star_1

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,0.023031,0.005641,0.014931,0.000704,0.038127,0.031396,0.008311,0.146612,0.000377,0.002398,...,0.004386,0.002187,0.640542,0.020804,0.001686,0.001267,0.00463,0.030688,0.010881,0.32229


In [ ]:
effect_A_star_1.loc[:,'IND']

category,final demand
Emissions,5.628171


In [ ]:
# change in the structure of intermediate inputs in countries other than India. 
effect_A_star_star = (1/2) * (((io_dict[2005].emissions.S @ io_dict[2014].L @ del_A_star_star @ io_dict[2005].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_star_star @ io_dict[2005].L @ io_dict[2005].Y)))
effect_A_star_star

region,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,CYP,CZE,...,PRT,ROU,ROW,RUS,SVK,SVN,SWE,TUR,TWN,USA
category,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,...,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand,final demand
Emissions,34.958443,2.488591,15.611332,-6.137231,40.922308,22.086449,8.46739,391.373811,1.513159,-2.908549,...,3.28038,-0.569019,558.729249,-125.71547,-1.084157,0.963519,3.77161,2.46485,23.926018,-303.121273


In [ ]:
effect_A_star_star.loc[:,'IND']

category,final demand
Emissions,17.07292


## **Consider an SDA with three time periods: 2000-2008, 2008-2011, 2011-2014**

# **4.3. Between 2000 and 2008**

* Repeating the above steps but now start year is 2000 and end year is 2008

In [ ]:
#Calculating required matrices

del_S_00_08 = io_dict[2008].emissions.S - io_dict[2000].emissions.S
del_L_00_08 = io_dict[2008].L - io_dict[2000].L
del_Y_00_08 = io_dict[2008].Y - io_dict[2000].Y
del_A_00_08 = io_dict[2008].A - io_dict[2000].A

indices = del_A_00_08.index.get_level_values("region") == "IND"
del_A_one_one_00_08 = pd.DataFrame(0, index=del_A_00_08.index, columns=del_A_00_08.columns)
del_A_one_one_00_08.loc[indices, indices] = del_A_00_08.loc[indices, indices]

ind_rows = del_A_00_08.index.get_level_values("region") == "IND"
other_rows = del_A_00_08.index.get_level_values("region") != "IND"
ind_cols = del_A_00_08.columns.get_level_values("region") == "IND"
other_cols = del_A_00_08.columns.get_level_values("region") != "IND"

del_A_one_star_00_08 = pd.DataFrame(0, index=del_A_00_08.index, columns=del_A_00_08.columns)
del_A_one_star_00_08.loc[ind_rows, other_cols] = del_A_00_08.loc[ind_rows, other_cols]
del_A_star_one_00_08 = pd.DataFrame(0, index=del_A_00_08.index, columns=del_A_00_08.columns)
del_A_star_one_00_08.loc[other_rows, ind_cols] = del_A_00_08.loc[other_rows, ind_cols]
del_A_star_star_00_08 = pd.DataFrame(0, index=del_A_00_08.index, columns=del_A_00_08.columns)
del_A_star_star_00_08.loc[other_rows, other_cols] = del_A_00_08.loc[other_rows, other_cols]

In [ ]:
# M&B SDA
intensity_effect_00_08 = (1/2) * (del_S_00_08 @ ((io_dict[2000].L @ io_dict[2000].Y) + (io_dict[2008].L @ io_dict[2008].Y)))
intensity_effect_00_08.loc[:, 'IND']

category,final demand
Emissions,-96.739953


In [ ]:
# M&B SDA
tech_effect_00_08 = (1/2) * ((io_dict[2000].emissions.S @ del_L_00_08 @ io_dict[2008].Y) + (io_dict[2008].emissions.S @ del_L_00_08 @ io_dict[2000].Y))
tech_effect_00_08.loc[:, 'IND']

category,final demand
Emissions,-70.321926


In [ ]:
# M&B SDA
dd_effect_00_08 = (1/2) * (((io_dict[2000].emissions.S @ io_dict[2000].L) + (io_dict[2008].emissions.S @ io_dict[2008].L)) @ del_Y_00_08)
dd_effect_00_08.loc[:, 'IND']

category,final demand
Emissions,635.135136


In [ ]:
# Further decomposing del_L
effect_A_one_one_00_08 = (1/2) * (((io_dict[2000].emissions.S @ io_dict[2008].L @ del_A_one_one_00_08 @ io_dict[2000].L @ io_dict[2008].Y) + (io_dict[2008].emissions.S @ io_dict[2008].L @ del_A_one_one_00_08 @ io_dict[2000].L @ io_dict[2000].Y)))
effect_A_one_one_00_08.loc[:, 'IND']

category,final demand
Emissions,-145.026639


In [ ]:
# Further decomposing del_L
effect_A_one_star_00_08 = (1/2) * (((io_dict[2000].emissions.S @ io_dict[2008].L @ del_A_one_star_00_08 @ io_dict[2000].L @ io_dict[2008].Y) + (io_dict[2008].emissions.S @ io_dict[2008].L @ del_A_one_star_00_08 @ io_dict[2000].L @ io_dict[2000].Y)))
effect_A_one_star_00_08.loc[:, 'IND']

category,final demand
Emissions,0.315443


In [ ]:
# Further decomposing del_L
effect_A_star_one_00_08 = (1/2) * (((io_dict[2000].emissions.S @ io_dict[2008].L @ del_A_star_one_00_08 @ io_dict[2000].L @ io_dict[2008].Y) + (io_dict[2008].emissions.S @ io_dict[2008].L @ del_A_star_one_00_08 @ io_dict[2000].L @ io_dict[2000].Y)))
effect_A_star_one_00_08.loc[:, 'IND']

category,final demand
Emissions,48.949191


In [ ]:
# Further decomposing del_L
effect_A_star_star_00_08 = (1/2) * (((io_dict[2000].emissions.S @ io_dict[2008].L @ del_A_star_star_00_08 @ io_dict[2000].L @ io_dict[2008].Y) + (io_dict[2008].emissions.S @ io_dict[2008].L @ del_A_star_star_00_08 @ io_dict[2000].L @ io_dict[2000].Y)))
effect_A_star_star_00_08.loc[:, 'IND']

category,final demand
Emissions,25.440079


# **4.4. Between 2008 and 2011**

* Repeating the above steps but now start year is 2008 and end year is 2011



In [ ]:
# Calculating required matrices

del_S_08_11 = io_dict[2011].emissions.S - io_dict[2008].emissions.S
del_L_08_11 = io_dict[2011].L - io_dict[2008].L
del_Y_08_11 = io_dict[2011].Y - io_dict[2008].Y
del_A_08_11 = io_dict[2011].A - io_dict[2008].A

indices = del_A_08_11.index.get_level_values("region") == "IND"
del_A_one_one_08_11 = pd.DataFrame(0, index=del_A_08_11.index, columns=del_A_08_11.columns)
del_A_one_one_08_11.loc[indices, indices] = del_A_08_11.loc[indices, indices]

ind_rows = del_A_08_11.index.get_level_values("region") == "IND"
other_rows = del_A_08_11.index.get_level_values("region") != "IND"
ind_cols = del_A_08_11.columns.get_level_values("region") == "IND"
other_cols = del_A_08_11.columns.get_level_values("region") != "IND"

del_A_one_star_08_11 = pd.DataFrame(0, index=del_A_08_11.index, columns=del_A_08_11.columns)
del_A_one_star_08_11.loc[ind_rows, other_cols] = del_A_08_11.loc[ind_rows, other_cols]
del_A_star_one_08_11 = pd.DataFrame(0, index=del_A_08_11.index, columns=del_A_08_11.columns)
del_A_star_one_08_11.loc[other_rows, ind_cols] = del_A_08_11.loc[other_rows, ind_cols]
del_A_star_star_08_11 = pd.DataFrame(0, index=del_A_08_11.index, columns=del_A_08_11.columns)
del_A_star_star_08_11.loc[other_rows, other_cols] = del_A_08_11.loc[other_rows, other_cols]

In [ ]:
# M&B SDA
intensity_effect_08_11 = (1/2) * (del_S_08_11 @ ((io_dict[2008].L @ io_dict[2008].Y) + (io_dict[2011].L @ io_dict[2011].Y)))
intensity_effect_08_11.loc[:, 'IND']

category,final demand
Emissions,-25.014507


In [ ]:
# M&B SDA
tech_effect_08_11 = (1/2) * ((io_dict[2008].emissions.S @ del_L_08_11 @ io_dict[2011].Y) + (io_dict[2011].emissions.S @ del_L_08_11 @ io_dict[2008].Y))
tech_effect_08_11.loc[:, 'IND']

category,final demand
Emissions,-14.00794


In [ ]:
# M&B SDA
dd_effect_08_11 = (1/2) * (((io_dict[2008].emissions.S @ io_dict[2008].L) + (io_dict[2011].emissions.S @ io_dict[2011].L)) @ del_Y_08_11)
dd_effect_08_11.loc[:, 'IND']

category,final demand
Emissions,404.852898


In [ ]:
# Further decomposing del_L
effect_A_one_one_08_11 = (1/2) * (((io_dict[2008].emissions.S @ io_dict[2011].L @ del_A_one_one_08_11 @ io_dict[2008].L @ io_dict[2011].Y) + (io_dict[2011].emissions.S @ io_dict[2011].L @ del_A_one_one_08_11 @ io_dict[2008].L @ io_dict[2008].Y)))
effect_A_one_one_08_11.loc[:, 'IND']

category,final demand
Emissions,-16.798218


In [ ]:
# Further decomposing del_L
effect_A_one_star_08_11 = (1/2) * (((io_dict[2008].emissions.S @ io_dict[2011].L @ del_A_one_star_08_11 @ io_dict[2008].L @ io_dict[2011].Y) + (io_dict[2011].emissions.S @ io_dict[2011].L @ del_A_one_star_08_11 @ io_dict[2008].L @ io_dict[2008].Y)))
effect_A_one_star_08_11.loc[:, 'IND']

category,final demand
Emissions,0.030477


In [ ]:
# Further decomposing del_L
effect_A_star_one_08_11 = (1/2) * (((io_dict[2008].emissions.S @ io_dict[2011].L @ del_A_star_one_08_11 @ io_dict[2008].L @ io_dict[2011].Y) + (io_dict[2011].emissions.S @ io_dict[2011].L @ del_A_star_one_08_11 @ io_dict[2008].L @ io_dict[2008].Y)))
effect_A_star_one_08_11.loc[:, 'IND']

category,final demand
Emissions,3.347502


In [ ]:
# Further decomposing del_L
effect_A_star_star_08_11 = (1/2) * (((io_dict[2008].emissions.S @ io_dict[2011].L @ del_A_star_star_08_11 @ io_dict[2008].L @ io_dict[2011].Y) + (io_dict[2011].emissions.S @ io_dict[2011].L @ del_A_star_star_08_11 @ io_dict[2008].L @ io_dict[2008].Y)))
effect_A_star_star_08_11.loc[:, 'IND']

category,final demand
Emissions,-0.587701


# **4.5. Between 2011 and 2014**

* Repeating the above steps but now start year is 2011 and end year is 2014

In [ ]:
# Calculating required matrices

del_S_11_14 = io_dict[2014].emissions.S - io_dict[2011].emissions.S
del_L_11_14 = io_dict[2014].L - io_dict[2011].L
del_Y_11_14 = io_dict[2014].Y - io_dict[2011].Y
del_A_11_14 = io_dict[2014].A - io_dict[2011].A

indices = del_A_11_14.index.get_level_values("region") == "IND"
del_A_one_one_11_14 = pd.DataFrame(0, index=del_A_11_14.index, columns=del_A_11_14.columns)
del_A_one_one_11_14.loc[indices, indices] = del_A_11_14.loc[indices, indices]

ind_rows = del_A_11_14.index.get_level_values("region") == "IND"
other_rows = del_A_11_14.index.get_level_values("region") != "IND"
ind_cols = del_A_11_14.columns.get_level_values("region") == "IND"
other_cols = del_A_11_14.columns.get_level_values("region") != "IND"

del_A_one_star_11_14 = pd.DataFrame(0, index=del_A_11_14.index, columns=del_A_11_14.columns)
del_A_one_star_11_14.loc[ind_rows, other_cols] = del_A_11_14.loc[ind_rows, other_cols]
del_A_star_one_11_14 = pd.DataFrame(0, index=del_A_11_14.index, columns=del_A_11_14.columns)
del_A_star_one_11_14.loc[other_rows, ind_cols] = del_A_11_14.loc[other_rows, ind_cols]
del_A_star_star_11_14 = pd.DataFrame(0, index=del_A_11_14.index, columns=del_A_11_14.columns)
del_A_star_star_11_14.loc[other_rows, other_cols] = del_A_11_14.loc[other_rows, other_cols]

In [ ]:
# M&B SDA
intensity_effect_11_14 = (1/2) * (del_S_11_14 @ ((io_dict[2011].L @ io_dict[2011].Y) + (io_dict[2014].L @ io_dict[2014].Y)))
intensity_effect_11_14.loc[:, 'IND']

category,final demand
Emissions,-75.534584


In [ ]:
# M&B SDA
tech_effect_11_14 = (1/2) * ((io_dict[2011].emissions.S @ del_L_11_14 @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ del_L_11_14 @ io_dict[2011].Y))
tech_effect_11_14.loc[:, 'IND']

category,final demand
Emissions,10.57988


In [ ]:
# M&B SDA
dd_effect_11_14 = (1/2) * (((io_dict[2011].emissions.S @ io_dict[2011].L) + (io_dict[2014].emissions.S @ io_dict[2014].L)) @ del_Y_11_14)
dd_effect_11_14.loc[:, 'IND']

category,final demand
Emissions,285.70363


In [ ]:
# Further decomposing del_L
effect_A_one_one_11_14 = (1/2) * (((io_dict[2011].emissions.S @ io_dict[2014].L @ del_A_one_one_11_14 @ io_dict[2011].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_one_one_11_14 @ io_dict[2011].L @ io_dict[2011].Y)))
effect_A_one_one_11_14.loc[:, 'IND']

category,final demand
Emissions,12.834617


In [ ]:
# Further decomposing del_L
effect_A_one_star_11_14 = (1/2) * (((io_dict[2011].emissions.S @ io_dict[2014].L @ del_A_one_star_11_14 @ io_dict[2011].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_one_star_11_14 @ io_dict[2011].L @ io_dict[2011].Y)))
effect_A_one_star_11_14.loc[:, 'IND']

category,final demand
Emissions,0.288091


In [ ]:
# Further decomposing del_L
effect_A_star_one_11_14 = (1/2) * (((io_dict[2011].emissions.S @ io_dict[2014].L @ del_A_star_one_11_14 @ io_dict[2011].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_star_one_11_14 @ io_dict[2011].L @ io_dict[2011].Y)))
effect_A_star_one_11_14.loc[:, 'IND']

category,final demand
Emissions,-15.971062


In [ ]:
# Further decomposing del_L
effect_A_star_star_11_14 = (1/2) * (((io_dict[2011].emissions.S @ io_dict[2014].L @ del_A_star_star_11_14 @ io_dict[2011].L @ io_dict[2014].Y) + (io_dict[2014].emissions.S @ io_dict[2014].L @ del_A_star_star_11_14 @ io_dict[2011].L @ io_dict[2011].Y)))
effect_A_star_star_11_14.loc[:, 'IND']

category,final demand
Emissions,13.428234


#**5. References**

* *Stadler, K., 2021. Pymrio – A Python Based Multi-Regional Input-Output Analysis Toolbox.  Journal of Open Research Software,  9(1), p.8.DOI: https://doi.org/10.5334/jors.251*

* *Timmer, M. P., Dietzenbacher, E., Los, B., Stehrer, R. and de Vries, G. J. (2015), "An Illustrated User Guide to the World Input–Output Database: the Case of Global Automotive Production" , Review of International Economics., 23: 575–605*

* *Wang, Z., Meng, J., Zheng, H., Shao, S., Wang, D., Mi, Z., & Guan, D. (2018). Temporal change in India’s imbalance of carbon emissions embodied in international trade. Applied energy, 231, 914-925.*

* *Kim, T. J., & Tromp, N. (2021). Analysis of carbon emissions embodied in South Korea's international trade: Production-based and consumption-based perspectives. Journal of Cleaner Production, 320, 128839.*

*  *Miller, R.E. and Blair, P.D. (2009) Input-Output Analysis: Foundations and Extensions. 2nd Edition, Cambridge University Press, New York. http://dx.doi.org/10.1017/CBO9780511626982*